## 本版本重要更改 
描述：提前映射成0-1 转化为0-1classification
1. missing=none
2. 完成cv
3. 完成一组参数的寻优 metric  
## 本版本目标 8000

## 当前情况更新（20210216）  
1. 关于auc 与 utility score的关系  auc基本维持不变，us会随着迭代次数的增加增长，但在LB上的表现不见好转  
2. 关于多阶段模型

## prepare the librarias used below

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import datatable as dt
from tqdm import tqdm

In [ ]:
!pip --quiet install ../input/treelite/treelite-0.93-py3-none-manylinux2010_x86_64.whl

In [ ]:
!pip --quiet install ../input/treelite/treelite_runtime-0.93-py3-none-manylinux2010_x86_64.whl

In [ ]:
# treelite
import treelite
import treelite_runtime 

# load data

In [ ]:
train_data_datatable = dt.fread('../input/jane-street-market-prediction/train.csv')
train_data = train_data_datatable.to_pandas()

In [ ]:
df=train_data.copy()

In [ ]:
# train_data['date']
# df[['weight','resp']].describe()

## Generally weights larger than 1 do correspond to leverage

In [ ]:
# df['weight'].value_counts()/len(df)
# 0.17 20%左右
#  为什么weight=0也会放上来
# 虽然weight=0 但是resp仍是准确的吧 室友有信息可用
# df[df['weight']==0]['resp'].plot(kind='hist')

In [ ]:
# len(df[(df['weight']==0) & (df['resp']>0)])/len(df[df['weight']==0])

In [ ]:
# len(df[df['resp']>0])/len(df)

In [ ]:
# len(df[(df['weight']!=0) & (df['resp']>0)])/len(df[df['weight']!=0])

In [ ]:
# # 差异显著性检验
# deltap=0.5044130406145096-0.5024071123449428
# p=0.504069666022587
# z=(deltap)/np.sqrt(p*(1-p)*(1/len(df[df['weight']==0])+1/len(df[df['weight']!=0])))
# z

In [ ]:
# 可见weight下确实有一些甄别作用
# import scipy.stats as st
# st.norm.cdf(z)

In [ ]:
# df['resp'].plot(kind='hist')
# 限定weight==0前后的resp分布基本一致 所以信息基本是有用的
# weight是什么？
# weight里面也是含有预判和信息的 所以可以理论上可以加入特征的行列

In [ ]:
# 内在逻辑是 对于weight>0的股票 我进行了交易，然后体现出来的resp是市面上的收益情况？-- 因为weight=0时同样有resp

In [ ]:
# len(df[(df['weight']>=0.5) & (df['resp']>0)])/len(df[df['weight']>=0.5])

In [ ]:
# len(df[(df['weight']>=1) & (df['resp']>0)])/len(df[df['weight']>=1])

In [ ]:
# 继续看下weight与resp的关联

In [ ]:
# 回归问题

In [ ]:
# 多目标怎样利用

In [ ]:
# xgb与NN的集成

# Simple pre_process for xgb

In [ ]:
## 缺失值暂不处理

In [ ]:
# np.percentile(df['weight'],[0,5,25,50,75,95,100])

## weight=0 but resp's info is ready to use

In [ ]:
## 暂时只取weight>0的部分
# df=df[df['weight']>0]

## generate ground truth action

In [ ]:
## 目前先直接根绝resp正负性转化为0-1问题
df['action']=df['resp'].apply(lambda x: 1 if x>0 else 0)

In [ ]:
# df['action'].value_counts()

1,0 还是比较均衡的，所以不存在样本不均衡的问题

In [ ]:
import copy
def add_features(df, features):
    new_features = copy.deepcopy(features)
    
    # todo
    df["cross_1_2"] = df["feature_1"] / (df["feature_2"] + 1e-5)
    df["cross_41_42_43"] = df["feature_41"] + df["feature_42"] + df["feature_43"]
    new_features.extend(["cross_1_2", "cross_41_42_43"])

    return df, new_features

In [ ]:
## 使用原始全部feature
features = [c for c in df.columns if 'feature' in c]
target = ['action']

In [ ]:
df,new_features=add_features(df,features)

In [ ]:
del train_data

# splitbydate:输入四个date节点拆分原始数据集为train and test

In [ ]:
# original dataframe;feature list;target list;and time point used to split dataset
# def splitbydate(oridf,feature,target,test_end,test_start,train_end,train_start=0): 
#     if 'date' not in oridf.columns:
#         print('Error!')
#         return
#     else:
#         feature_=feature.copy()
#         target_=target.copy()
#         feature_.append('date')
#         target_.append('date')
#         #print(target_)
#         x_df=oridf[feature_]
#         y_df=oridf[target_]
#         X_train,X_test=x_df[(x_df['date']>=train_start) & (x_df['date']<train_end)],x_df[(x_df['date']>=test_start) & (x_df['date']<test_end)]
#         y_train,y_test=y_df[(y_df['date']>=train_start) & (y_df['date']<train_end)],y_df[(y_df['date']>=test_start) & (y_df['date']<test_end)]
#         vali=oridf[["date","weight","resp","action"]][(oridf['date']>=test_start) & (oridf['date']<test_end)]
#         return X_train[feature],X_test[feature],y_train[target],y_test[target],vali

In [ ]:
# original dataframe;feature list;target list;and time point used to split dataset
def splitbydate_more(oridf,feature,target,test_end,test_start,train_end,train_start=0): 
    if 'date' not in oridf.columns:
        print('Error!')
        return
    else:
        feature_=feature.copy()
        target_=target.copy()
        feature_.append('date')
        target_.append('date')
        #print(target_)
        x_df=oridf[feature_]
        y_df=oridf[target_]
        X_train,X_test=x_df[(x_df['date']>=train_start) & (x_df['date']<train_end)],x_df[(x_df['date']>=test_start) & (x_df['date']<test_end)]
        y_train,y_test=y_df[(y_df['date']>=train_start) & (y_df['date']<train_end)],y_df[(y_df['date']>=test_start) & (y_df['date']<test_end)]
        vali=oridf[["date","weight","resp","action"]][(oridf['date']>=test_start) & (oridf['date']<test_end)]
        trai=oridf[["date","weight","resp","action"]][(oridf['date']>=train_start) & (oridf['date']<train_end)]
        return X_train[feature],X_test[feature],y_train[target],y_test[target],trai,vali

In [ ]:
# X_train,X_test,y_train,y_test,trai,vali=splitbydate_more(df,features,target,500,450,400,85)

# xgboost参数
https://www.kaggle.com/code1110/janestreet-faster-inference-by-xgb-with-treelite

In [ ]:
def utility_scoring(df):
    """
    To get the utility score used in the challenge.
    """
    from math import sqrt
    u = 0
    Pi = []
    Pis = []
    count_i = len(df['date'].unique())
    for i in list(df['date'].unique()):
        #print("date value= ", i)
        #print(10*"=")
        tmp = df[df['date'] == i][["date","weight","resp","action"]]
        tmp["mult"] = tmp['weight'] * tmp['resp'] * tmp['action']
        Pi.append(tmp["mult"].sum())
        Pis.append((tmp["mult"].sum())**2)
    
    t =  sum(Pi)/sqrt(sum(Pis)) * sqrt(250/count_i)
    u = min(max(t,0),6)*sum(Pi)
    return u

In [ ]:
thres=0.5

In [ ]:
import xgboost as xgb

In [ ]:
# we need/have
# 并不涉及到与原始真实值的比较 只是自顾自得输出
# 先弄一个绝对评分的函数- 对比得分与auc表现的一致性
# 可以弄一个指标 例如预测得分/真实得分

# trai vai
def utility_score(predt,dtrain):
    """
    To get the utility score used in the challenge.
    """
    y = dtrain.get_label()
    tarlen=len(y)
    if tarlen==len(trai):
        u=utility_scoring(trai)
    elif tarlen==len(vali):
        u=utility_scoring(vali)
#     vali['actionv'] = (predt> 0.5)*1
    
    return 'utilityscore',float(u)

# trai vai
def relative_utility_score(predt,dtrain):
    """
    To get the utility score used in the challenge.
    """
    
    y = dtrain.get_label()
    tarlen=len(y)
    if tarlen==len(trai):
        trai['action']=(predt> thres)*1
        u2=utility_scoring(trai)
        u=u2/trai_ori
    elif tarlen==len(vali):
        vali['action']=(predt> thres)*1
        u2=utility_scoring(vali)
        u=u2/vali_ori
#     vali['actionv'] = (predt> 0.5)*1
    
    return 'r_utilityscore',float(u)

In [ ]:
# params['learning_rate']=0.05
# params

In [ ]:
# dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
# dvalid = xgb.DMatrix(X_test.values, label=y_test.values)
# bst = xgb.train(params,
#                 dtrain,
#                 feval=utility_score,
#                 num_boost_round=500,
#                 evals=[(dtrain, 'train'), (dvalid, 'eval')],
#                 early_stopping_rounds=20)

In [ ]:
# trai_ori=utility_scoring(trai)
# vali_ori=utility_scoring(vali)

In [ ]:
import gc
gc.enable()

In [ ]:
# early_stop = xgb.callback.EarlyStopping(rounds=10,
#                                     metric_name='r_utilityscore',
#                                     data_name='eval')

In [ ]:
SEED = 21
params = {
    #general parameter
    
    'booster':'gbtree',
    
    #tree parameter
    
    # shrink the weight to make the process more conservative
    'learning_rate': 0.05,
    # minimum loss reduce needed to make a new partition
    'gamma':0,
    # max depth of the tree
    'max_depth': 8,
    'min_child_weight':10,
    'tree_method': 'gpu_hist', # Let's use GPU for a faster experiment
    # 随机采样
    'colsample_bytree': 0.72,                 
    'subsample': 0.8,
    
    'seed': SEED,
    
    #print every 10 times
#     'verbose_eval':100,
   
    #learning task parameter
    # 可考虑更换目标函数为auc
    'objective':'binary:logistic',
    'disable_default_eval_metric': 1
#     'eval_metric':['auc']
#     'n_boost_round': 600,
#     'early_stopping_rounds':20
    

}

In [ ]:
# results={}

In [ ]:
# dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
# dvalid = xgb.DMatrix(X_test.values, label=y_test.values)

# bst = xgb.train(params,
#                 dtrain,
#                 evals=[(dtrain, 'train'), (dvalid, 'eval')],
#                 feval=relative_utility_score,
#                 num_boost_round=50,
#                 callbacks=[early_stop]
# #                 evals_result=results
# #                 early_stopping_rounds=10
#                )

In [ ]:
# import matplotlib.pyplot as plt

In [ ]:
# plt.figure(figsize=(15,10))
# plt.plot(np.arange(1000),results['train']['r_utilityscore'])
# plt.plot(np.arange(1000),results['eval']['r_utilityscore'])

In [ ]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(20,15)) 
# xgb.plot_importance(bst,ax=plt.gca())

In [ ]:
# fscore=pd.DataFrame.from_dict(bst.get_fscore(),orient='index')
# fscore.columns=['fs']
# fscore=fscore.sort_values(by=['fs'],ascending=False)
# # fscore

In [ ]:
# plt.figure(figsize=(15,10))
# plt.bar(fscore.index,fscore['fs'].values)

In [ ]:
# fscore[fscore['fs']>1000]

In [ ]:
# print("# original features: {}".format(len(features)))
# print("# features included in xgb model: {}".format(len(fscore)))

In [ ]:
# import re
# def cutnum(x):
#     res=re.match(r'([a-z]*)([0-9]*)',x).group(2)
#     return int(res)

In [ ]:
# one feature has none improvement on prediction
# select top 60% feature to service for final training
# fscore['num_f']=fscore.index.map(cutnum)
# fscore_se=fscore[:90]
# fscore_se

In [ ]:
# f_feature=[]
# for i in fscore_se['num_f']:
# #     print(i)
#     f_feature.append(features[i])
# f_feature

In [ ]:
# new_features

In [ ]:
# del X_train,X_test,y_train,y_test,trai,vali,trai_ori,vali_ori,dtrain,dvalid

In [ ]:
X_train,X_test,y_train,y_test,trai,vali=splitbydate_more(df,new_features,target,500,500,85+415,85)
dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
trai_ori=utility_scoring(trai)
# vali_ori=utility_scoring(vali)
# dvalid = xgb.DMatrix(X_test.values, label=y_test.values)
bst = xgb.train(params,
        dtrain,
        num_boost_round=600)

In [ ]:
# subtree_depth_params = [
#     (max_depth, min_child_weight)
#     for max_depth in range(8,13)
#     for min_child_weight in range(7,10)
# ]

In [ ]:
# subtree_depth_params

In [ ]:
# cv_times=5
# start_list=[90,110,130,150,170]

In [ ]:
# import gc
# gc.enable()

# del train_data

In [ ]:
# bestscore=0
# bestparams=params
# cv_score=[]

In [ ]:
# def tune_parameters(params,params_valuelist):
#     for max_depth, min_child_weight in params_valuelist:
#         print("CV with max_depth={}, min_child_weight={}".format(
#                                  max_depth,
#                                  min_child_weight))
#         # Update our parameters
#         params['max_depth'] = max_depth
#         params['min_child_weight'] = min_child_weight
#         score=[]
#         for i in range(cv_times):
#             # train length 250
#             # test legth 50
#             # 500 450 420 170
#             # 480 430 400 150
#             # 460 410 380 130
#             # 110
#             # 90
#             X_train,X_test,y_train,y_test,vali=splitbydate(df,features,target,start_list[i]+330,start_list[i]+280,start_list[i]+250,start_list[i])
#             dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
#             dvalid = xgb.DMatrix(X_test.values, label=y_test.values)
#             bst = xgb.train(params,
#                     dtrain,
#                     num_boost_round=500,
#                     evals=[(dtrain, 'train'), (dvalid, 'eval')],
#                     early_stopping_rounds=20,
#                     verbose_eval=100)
#             score.append(bst.best_score)
#             del X_train,X_test,y_train,y_test,vali

#         cv_score.append(np.mean(score))
#     #     print("CV with max_depth={}, min_child_weight={}".format(
#     #                              max_depth,
#     #                              min_child_weight))
#         if np.mean(score)>bestscore:
#     #         bestparams=params
#             bestscore=np.mean(score)
#             best_max_depth,best_min_child_weight=max_depth,min_child_weight

#     bestparams['max_depth'] = max_depth
#     bestparams['min_child_weight'] = min_child_weight
#     bestparams

In [ ]:
# for max_depth, min_child_weight in subtree_depth_params:
#     print("CV with max_depth={}, min_child_weight={}".format(
#                              max_depth,
#                              min_child_weight))
#     # Update our parameters
#     params['max_depth'] = max_depth
#     params['min_child_weight'] = min_child_weight
#     score=[]
#     for i in range(cv_times):
#         # train length 250
#         # test legth 50
#         # 500 450 420 170
#         # 480 430 400 150
#         # 460 410 380 130
#         # 110
#         # 90
#         X_train,X_test,y_train,y_test,vali=splitbydate(df,features,target,start_list[i]+330,start_list[i]+280,start_list[i]+250,start_list[i])
#         dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
#         dvalid = xgb.DMatrix(X_test.values, label=y_test.values)
#         bst = xgb.train(params,
#                 dtrain,
#                 num_boost_round=500,
#                 evals=[(dtrain, 'train'), (dvalid, 'eval')],
#                 early_stopping_rounds=20,
#                 verbose_eval=100)
#         score.append(bst.best_score)
#         del X_train,X_test,y_train,y_test,vali
    
#     cv_score.append(np.mean(score))
# #     print("CV with max_depth={}, min_child_weight={}".format(
# #                              max_depth,
# #                              min_child_weight))
#     if np.mean(score)>bestscore:
# #         bestparams=params
#         bestscore=np.mean(score)
#         best_max_depth,best_min_child_weight=max_depth,min_child_weight
        
# bestparams['max_depth'] = max_depth
# bestparams['min_child_weight'] = min_child_weight
# bestparams

In [ ]:
# score

In [ ]:
# import matplotlib.pyplot as plt

In [ ]:
# plt.plot([str(x) for x in subtree_depth_params],cv_score)

In [ ]:
# bestparams

In [ ]:
# bestscore

In [ ]:
# subtree_depth_params = [
#     (max_depth, min_child_weight)
#     for max_depth in range(6,10)
#     for min_child_weight in range(8,12)
# ]

In [ ]:
# del X_train,X_test,y_train,y_test,trai,vali,trai_ori,vali_ori,dtrain,dvalid

In [ ]:
import gc
gc.enable()

In [ ]:
# bestscore=0
# bestparams=params
# cv_score=[]
# for lr in [.3, .2, .1, .05, .01, .005]:
#     print("learning rate={}".format(
#                              lr))
#     # Update our parameters
#     params['learning_rate'] = lr
#     score=[]
#     for i in range(cv_times):
#         X_train,X_test,y_train,y_test,trai,vali=splitbydate_more(df,f_feature,target,start_list[i]+330,start_list[i]+280,start_list[i]+250,start_list[i])
#         trai_ori=utility_scoring(trai)
#         vali_ori=utility_scoring(vali)
#         dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
#         dvalid = xgb.DMatrix(X_test.values, label=y_test.values)
#         bst = xgb.train(params,
#                 dtrain,
#                 num_boost_round=500,
#                 feval=relative_utility_score,
#                 evals=[(dtrain, 'train'), (dvalid, 'eval')],
#                 early_stopping_rounds=20,
#                 verbose_eval=100)
#         score.append(bst.best_score)
#         del X_train,X_test,y_train,y_test,trai,vali,trai_ori,vali_ori,dtrain,dvalid
    
#     cv_score.append(np.mean(score))
# #     print("CV with max_depth={}, min_child_weight={}".format(
# #                              max_depth,
# #                              min_child_weight))
#     if np.mean(score)>bestscore:
#         bestparams=params
#         bestscore=np.mean(score)

In [ ]:
# cv_score

In [ ]:
# plt.plot([str(x) for x in subtree_depth_params],cv_score)
# plt.xticks(rotation=60)

In [ ]:
# bestparams['max_depth'] = 6
# bestparams['min_child_weight'] = 10
# bestparams

In [ ]:
# del X_train,X_test,y_train,y_test,vali

In [ ]:
# X_train,X_test,y_train,y_test,vali=splitbydate(df,features,target,500,470,440,85)
# dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
# dvalid = xgb.DMatrix(X_test.values, label=y_test.values)
# bst = xgb.train(bestparams,
#                 dtrain,
#                 num_boost_round=500,
#                 evals=[(dtrain, 'train'), (dvalid, 'eval')],
#                 early_stopping_rounds=20)

In [ ]:
# 基于treelite进行编译
model = treelite.Model.from_xgboost(bst)

In [ ]:
# generate shared library
toolchain = 'gcc'
model.export_lib(toolchain=toolchain, libpath='./mymodel_score_600rounds_fullfeature_1.so',
                 params={'parallel_comp': 32}, verbose=True)

In [ ]:
# predictor from treelite
# predictor = treelite_runtime.Predictor('./mymodel_score.so', verbose=True)

In [ ]:
# dtest=xgb.DMatrix(X_test.values, label=y_test.values)

In [ ]:
# def utility_scoring(df):
#     """
#     To get the utility score used in the challenge.
#     """
#     from math import sqrt
#     u = 0
#     Pi = []
#     Pis = []
#     count_i = len(df['date'].unique())
#     for i in list(df['date'].unique()):
#         #print("date value= ", i)
#         #print(10*"=")
#         tmp = df[df['date'] == i][["date","weight","resp","actionv"]]
#         tmp["mult"] = tmp['weight'] * tmp['resp'] * tmp['actionv']
#         Pi.append(tmp["mult"].sum())
#         Pis.append((tmp["mult"].sum())**2)
    
#     t =  sum(Pi)/sqrt(sum(Pis)) * sqrt(250/count_i)
#     u = min(max(t,0),6)*sum(Pi)
#     return u

In [ ]:
# treelite
# batch = treelite_runtime.Batch.from_npy2d(X_test.values)
# predicted_treelite = predictor.predict(batch)

In [ ]:
# predicted_treelite

In [ ]:
# from sklearn.metrics import log_loss
# score = log_loss(y_test.values, predicted_treelite)
# print(score)

In [ ]:
# best_i = 0
# best_u = 0
# for i in [0.41, 0.45,0.49,0.5, 0.51,0.52, 0.55, 0.6,0.63, 0.65]:
    
#     vali['action'] = (predictor.predict(batch) > i)*1
#     u = utility_scoring(vali)
#     print(u)
#     if u > best_u:
#         best_u = u
#         best_i = i

In [ ]:
# import janestreet
# env = janestreet.make_env() # initialize the environment
# iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
# threshold=best_i

In [ ]:
# best_i

In [ ]:
# for (test_df, pred_df) in tqdm(iter_test):
#     if test_df['weight'].item() > 0:
#         # inference with treelite
#         batch = treelite_runtime.Batch.from_npy2d(test_df.loc[:, features].values)
#         pred_df.action = (predictor.predict(batch) > threshold).astype('int')
#     else:
#         pred_df.action = 0
#     env.predict(pred_df)